In [ ]:
#spark
__author__ = 'vcoder'

import os
import sys

#change them when spark and pyspark path changes

#SPARK_HOME = "/home/vcoder/EDA/spark-1.5.0"
#SPARK_HOME_PYTHON = SPARK_HOME + "/python"

SPARK_HOME = "/home/worker/software/spark-1.5.0"
SPARK_HOME_PYTHON = SPARK_HOME + "/python"

os.environ['SPARK_HOME'] = SPARK_HOME
sys.path.append(SPARK_HOME_PYTHON)

from pyspark import SparkContext
from pyspark import SparkConf

import geoip2.database
reader = geoip2.database.Reader('GeoLite2-City.mmdb')

def ip2city(ip):
    try:
        city = reader.city(ip).city.name
    except:
        city = 'not found'
    return city

def ip2la(ip):
    try:
        la = reader.city(ip).location.latitude
    except:
        la = 0.
    return la

def ip2lo(ip):
    try:
        lo = reader.city(ip).location.longitude
    except:
        lo = 0.
    return lo


In [ ]:
sc = SparkContext('local', 'ip2geo')

In [ ]:
normalPath = 'csv/output.csv'
attackPath = 'csv/output.csv'

In [ ]:
normalRaw = sc.textFile(normalPath).map(lambda x: x.split())
attackRaw = sc.textFile(attackPath).map(lambda x: x.split())

normalMax = normalRaw.map(lambda x: x[0]).max()
normalMin = nromalRaw.map(lambda x: x[0]).min()
attackMax = attackRaw.map(lambda x: x[0]).max()
attackMin = attackRaw.map(lambda x: x[0]).min()

normalIp = normalRaw.map(lambda x: [x[1], x[2], x[0]*100/(normalMax - normalMin)])
attackIp = attackRaw.map(lambda x: [x[1], x[2], x[0]*100/(attackMax - attackMin)])

normalSrc = normalRaw.map(lambda x: x[1])
normalDst = normalDst.map(lambda x: x[2])
attackSrc = attackRaw.map(lambda x: x[1])
attackDst = attackDst.map(lambda x: x[2])

allIPs = normalSrc.union(normalDst).union(attackSrc).union(attackDst).distinct().collect()


In [ ]:
geoData = []
for ip in allIPs:
    lo = ip2lo(ip)
    la = ip2la(ip)
    geoData.append({'ip' : ip,
                    'longtitude' : lo,
                    'lantitude' : la})

In [ ]:
attackNum = 1000
normalNum = 1000
victimNum = 1000

attackDS = []
for x in attackIps.take(attackNum):
    src = x[0]
    dst = x[1]
    val = x[2]
    attackDS .append({'source': src,
                      'destination' : dst,
                      'value' : val})
    
normalDS = []
for x in normalIps.take(normalNum):
    src = x[0]
    dst = x[1]
    val = x[2]
    normalDS .append({'source': src,
                      'destination' : dst,
                      'value' : val})

In [ ]:
victimeData = []
victimIPs = attackIps.takeOrdered(victimNum, key = lambda x: -x[2])
for x in victimeIPs:
    dst = x[1]
    val = x[2]
    victimData.append({'destination' : dst,
                       'value' :val})

In [ ]:
ipDistribution = {'geoData' : geoData,
                  'attackDS' : attackDS,
                  'normalDS' : normalDS,
                  'victimData' : victimData}

In [ ]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

dbName = 'test'
collectionName = 'ipDist'

db = client[dbName]
collection = db[collectionName]

collection.insert_one(ipDistribution)

In [ ]:
sc.stop()